In [ ]:
%cd M:/Rafael/Documentos/GitHub/TCC
import json
import numpy as np
from datetime import datetime
from vector import recupera_arquivos_token
from tqdm import tqdm

In [ ]:
def carrega_dados(filtro=None):
    ''' Importa arquivos dos dados dos artigos ''' 
    dados = {}
    lista_arquivos_entrada = recupera_arquivos_token(filtro=filtro) 
    for nome_arquivo in lista_arquivos_entrada:
        caminho = 'data/dataset_final/'
        hora = datetime.now().strftime("%H:%M:%S")
        print(f'({hora}) Processando arquivo {nome_arquivo} ... ', end='')
        with open(caminho + nome_arquivo, 'r', encoding = 'utf-8') as arquivo_json:
            artigos = json.load(arquivo_json)
            nome_portal = nome_arquivo[2:-5].split('-')[0]
            if not nome_portal in dados:
                dados[nome_portal] = artigos
                #print('\ncriando', len(artigos))
            else:
                dados[nome_portal].extend(artigos)
            print(f'{len(artigos)} artigos carregados ')
    return dados, filtro or 'Todos'

#dados_e = carrega_dados('E')    
#dados_d = carrega_dados('D')    
#dados_i = carrega_dados('I') 

#'247'
#'dcm'
#'conexaopolitica'
#'tercalivre'
#'senso'
#'republica'


dados, portal = carrega_dados()
    

In [ ]:
portal

In [ ]:
# Total de registros
len_total = 0 
for dado in dados:
    print(dado, '\t' * round((3 - len(dado)/5)), len(dados[dado]))
    len_total += len(dados[dado])
print(f'TOTAL\t\t {len_total}')    

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def plota_duplicados(portal):
    ''' Representação em gráfico de torta ou pizza da quantidade de valores duplucados para um determinado portal '''
    df = pd.DataFrame(dados[portal])
    perc_duplicados = round((df.duplicated().value_counts()[1] / df.duplicated().count()) * 100, 1)
    
    df.duplicated().value_counts().plot.pie(title=f'Portal: {portal}', ylabel='', 
                                            labels=['',f'duplicados {perc_duplicados}%'])
    
    plt.savefig("dup_" + portal +".png")

In [ ]:
plota_duplicados('conexaopolitica')

In [ ]:
plota_duplicados('republica')    

In [ ]:
plota_duplicados('senso')

In [ ]:
plota_duplicados('tercalivre')

In [ ]:
plota_duplicados('247')

In [ ]:
plota_duplicados('dcm')

In [ ]:
# Levantamento de informações de armazenamento do dataframe (Memória/Min/Max)

df_info = pd.DataFrame(columns=['Portal', 'Memória (GB)', 'Documentos', 'Palavras', '(min)', '(max)'])
df_info = df_info.astype({'Portal':str, 'Memória (GB)':float, 'Documentos':int, 'Palavras':int, '(min)':int, '(max)':int })      

GB = 1000 * 1000 * 1000
for portal in dados:
    df = pd.DataFrame(dados[portal])
    tamanho_df = round(df.memory_usage().sum() / GB, 2)
    n_palavras = 0
    min_palavras = 99999
    max_palavras = 0
    for documento in dados[portal]:
        n_palavras += len(documento)
        if len(documento) < min_palavras:
            min_palavras = len(documento)
        if len(documento) > max_palavras:
            max_palavras = len(documento)    
    info = [{'Portal':portal, 'Memória (GB)':tamanho_df,  'Documentos':len(dados[portal]), 
             'Palavras':n_palavras, '(min)':min_palavras, '(max)':max_palavras}]
    df_info = df_info.append(info, ignore_index=True)
    print(portal, '- Memória ocupada:', str(tamanho_df), 'GB', min_palavras, max_palavras)
    

In [ ]:
df_info.round(2)

In [ ]:
df_info.round(2).loc[: , ['Memória (GB)', 'Documentos' , 'Palavras']].sum()

In [ ]:
# Verificando outliers
import pandas as pd
import time
import matplotlib.pyplot as plt

fig, axes = plt.subplots(nrows=1, ncols=1)

len(dados)
lista_wordcount = []
for dado in dados:
    artigos = dados[dado]
    for artigo in artigos:
        lista_wordcount.append(len(artigo))

serie = pd.Series(lista_wordcount)

df_describe = pd.DataFrame(columns=serie.describe().index)
df_describe = df_describe.append(serie.describe(), ignore_index=True)
df_describe.columns = ['Nº Registros', 'Média', 'Desvio padrão', 'Min',
                       '25%', '50%', '75%', 'Max']

serie.plot.hist( figsize=(14,4),  title='Contagem de palavras x documentos' , bins=300)
axes.set_xlabel('nº palavras')
axes.set_ylabel('documentos')

df_describe.style.hide_index().format('{:.2f}')


In [ ]:
serie.plot.box(showfliers=False ,  title='Boxplot contagem de palavras') 
axes.set_ylabel('nº palavras')

In [ ]:
serie

In [ ]:
# Encontra uma linha de corte - Calcula o Escore Z e busca o maior valor da série, onde z < 3.5
from scipy import stats

z = np.abs(stats.zscore(serie))
n_corte = serie[np.where(z < 3.5)[0]].max()
print(f'n_corte ({n_corte})')

In [ ]:
# Criar DF (COM SLICING) sem importar denovo os arquivos (pegando o q ta na memoria)
import pandas as pd

df = None
for dado in dados:
    print(f'Carregando {dado} ...')
    artigos = dados[dado]
    tam = len(artigos)
    
    df_temp = pd.DataFrame(artigos).iloc[:, :n_corte]
    
    df_temp['portal'] = dado
    if df is None:
        df = df_temp
    else:        
        df = df_temp.append(df, ignore_index=True)
        


In [ ]:
# Lista a quantidade de documentos agrupados por Site, antes da remoção dos duplicados
df.groupby(df['portal']).size()

In [ ]:
df.shape

In [ ]:
# Remove registros duplicados
df.drop_duplicates(ignore_index=True, inplace=True)

In [ ]:
# Lista a quantidade de documentos agrupados por Site, após a remoção dos duplicados
df.groupby(df['portal']).size()

In [ ]:
df.shape

In [ ]:
plot_1 = df.groupby(df['portal']).size().plot.bar(xlabel = 'Portais' ,ylabel='Nº documentos')

In [ ]:
df.loc[((df.portal=='247') | (df.portal=='dcm'), 'posicao')] = 'Esquerda'
df.loc[(~(df.portal=='247') & ~(df.portal=='dcm'), 'posicao')] = 'Direita'

In [ ]:
df.groupby(df['posicao']).size()

In [ ]:
# Nuvem de palavras
def carrega_dados_wordcloud_portal(df, portal=None):
    texto = df.loc[df['portal']==portal].drop(axis=1,labels=['portal', 'posicao']).fillna(value=np.nan).to_string(header=False, index=False, na_rep="")
    return texto


# Define a função plotar nuvem de palavras
def plot_cloud(wordcloud, titulo):
    # Define tamanho
    plt.figure(figsize=(40, 30))
    # Mostra imagem
    plt.title(f'wordcloud: \'{titulo}\'')
    plt.imshow(wordcloud) 
    # Sem detalhes de axis
    plt.axis("off");
    plt.savefig(f'nuvem_{titulo}.png')
    

from wordcloud import WordCloud
import matplotlib.pyplot as plt

%matplotlib inline

texto = ""
texto_esq = ""
texto_dir = ""
for p in df['portal'].unique():
    print(f'Gerando nuvem de palavras para \'{p}\' ...')    
    texto = carrega_dados_wordcloud_portal(df, portal=p)

    if p == '247' or p == 'dcm':
        texto_esq += texto
    else:
        texto_dir += texto
        
    # Gera Nuvem de Palavras
    wordcloud = WordCloud(width = 3000, height = 2000, random_state=1, background_color='white', 
                          colormap='bone', normalize_plurals=False, regexp=r'[\w\-]+').generate(texto)

    # Plota as imagens
    plot_cloud(wordcloud, p)

    

In [ ]:
p = 'direita'

# Gera Nuvem de Palavras
wordcloud = WordCloud(width = 3000, height = 2000, random_state=1, background_color='white', 
                      colormap='bone', normalize_plurals=False, regexp=r'[\w\-]+').generate(texto_dir)
# Plot
plot_cloud(wordcloud, p)

In [ ]:
p = 'esquerda'

# Gera Nuvem de Palavras
wordcloud = WordCloud(width = 3000, height = 2000, random_state=1, background_color='white', 
                      colormap='bone', normalize_plurals=False, regexp=r'[\w\-]+').generate(texto_esq)
# Plot
plot_cloud(wordcloud, p)

In [ ]:
p = 'Todos'

# Gera Nuvem de Palavras
wordcloud = WordCloud(width = 3000, height = 2000, random_state=1, background_color='white', 
                      colormap='bone', normalize_plurals=False, regexp=r'[\w\-]+').generate(texto_esq + " " + texto_dir)
# Plot
plot_cloud(wordcloud, p)


In [ ]:
plot_2 = df.groupby(df['posicao']).size().plot.bar(xlabel = 'Posição' ,ylabel='Nº documentos')

In [ ]:
contagem_palavras = df.drop(axis=1,labels=['portal', 'posicao']).stack().value_counts() 
len(contagem_palavras)

In [ ]:
contagem_palavras.head(30).iloc[::-1].plot.barh(figsize=(8,14),  zorder=2, width=0.9);

In [ ]:
df

In [ ]:
df.to_csv('dataset_final.csv', encoding='utf-8',sep=';', index=False)